# RQ 1: Did the 2012 Olympic games increase tourist activity in East London region?

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime

In [2]:
# Importing data
df = pd.read_csv('UK_international-visits.csv')
df.head()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_55602/3518813486.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('UK_international-visits.csv')


,year,quarter,market,dur_stay,mode,purpose,area,visits,spend,nights,sample
0,2002,January-March,Belgium,1-3 nights,Air,Holiday,TOTAL ENGLAND,4.431616,1.013484,9.530416,6
1,2002,January-March,Belgium,1-3 nights,Air,Holiday,LONDON,3.572186,0.969138,6.954456,5
2,2002,January-March,Belgium,1-3 nights,Air,Holiday,REST OF ENGLAND,0.859430,0.044346,2.575960,1
3,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTHERN ENGLAND,0.859430,0.044346,2.575960,1
4,2002,January-March,Belgium,1-3 nights,Air,Holiday,SOUTH EAST,0.859430,0.044346,2.575960,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549853 entries, 0 to 549852
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   year      549853 non-null  object 
 1   quarter   549853 non-null  object 
 2   market    549853 non-null  object 
 3   dur_stay  549853 non-null  object 
 4   mode      549853 non-null  object 
 5   purpose   549853 non-null  object 
 6   area      549853 non-null  object 
 7   visits    549853 non-null  float64
 8   spend     549853 non-null  float64
 9   nights    549853 non-null  float64
 10  sample    549853 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 46.1+ MB


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Assuming you have a dataframe called 'df' and a list of columns to normalize
columns_to_normalize = ['visits','spend','nights', 'sample']

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
df[columns_to_normalize].head()

,visits,spend,nights,sample
0,0.019436,0.002407,0.002366,0.014925
1,0.015667,0.002302,0.001726,0.011940
2,0.003769,0.000108,0.000639,0.000000
3,0.003769,0.000108,0.000639,0.000000
4,0.003769,0.000108,0.000639,0.000000


In [5]:
#import sav files for cultural engagement and concatenating them
df = pd.read_spss('spss12/1a.sav')
df.head()

,scrser,cserial,cscreen,dwell1,dwell2,dwellin,nadults,nwadult,sex1,sex2,...,padd1,pind1,cdweight1,pind3,cdweight3,nrweight2,finalwt1,finalwt3,popwt1,popwt3
0,1001010.0,100101.0,Main Sample Address,1.0,NaN,1.0,1.0,NaN,Male,NaN,...,0.004058,0.004058,0.310474,0.004058,0.347056,1.613,0.262512,0.284713,399.943051,402.646790
1,1001030.0,100103.0,Main Sample Address,1.0,NaN,1.0,3.0,NaN,Male,Female,...,0.004058,0.001353,0.931421,0.001353,1.041167,1.613,0.812436,0.895140,1237.766607,1265.926032
2,1001040.0,100104.0,Main Sample Address,1.0,NaN,1.0,2.0,NaN,Female,Male,...,0.004058,0.002029,0.620947,0.002029,0.694112,1.704,0.529924,0.568226,807.351705,803.596438
3,1001050.0,100105.0,Main Sample Address,1.0,NaN,1.0,4.0,NaN,Male,Male,...,0.004058,0.001015,1.241894,0.001015,1.388223,1.818,1.425664,1.552870,2172.033573,2196.100930
4,1001080.0,100108.0,Main Sample Address,1.0,NaN,1.0,2.0,NaN,Male,Female,...,0.004058,0.002029,0.620947,0.002029,0.694112,1.704,0.546224,0.595327,832.186264,841.923547


In [6]:
df.shape

(28117, 1833)

In [7]:
df1 = pd.read_spss('spss12/1c.sav')
#print all rows where gor6 != "Excluded from filter"
df1 = df1[df1['gor6'] != "Excluded from filter"]
df1.head()
#print gor6 categories
df1['gor6'].unique()
df1['gor6'].value_counts()

North                   457
South                   322
Midlands                303
London                  222
East                    136
Excluded from filter      0
Name: gor6, dtype: int64

In [8]:
#rename muspsa column to museum_visits
df1.rename(columns={'muspsa':'museum_visits'}, inplace=True)
#convert museum_visits column by mapping
# ['Been to 1+ museum/gallery in last year': 1, 'No associated adult data': 0, 'Not been to museum/gallery in last year': 0]
df1['museum_visits'] = df1['museum_visits'].map({'Been to 1+ museum/gallery in last year': 1, 'No associated adult data': 0, 'Not been to museum/gallery in last year': 0})

df1["muspsa2"] = df1["museum_visits"].map({'Been to 1+ museum/gallery in last year': 1, 'No associated adult data': 0, 'Not been to museum/gallery in last year': 0})

#sum both columns together to get total number of museum visits
df1['museum_visits'] = df1['museum_visits'] + df1['muspsa2']
#drop muspsa2 column
df1.drop(['muspsa2'], axis=1, inplace=True)



In [9]:
heritage_site_cols = ['herpsa', 'herpsa2']
